# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846863471452                   -0.70    4.2
  2   -7.852322594777       -2.26       -1.53    1.0
  3   -7.852646005588       -3.49       -2.53    3.0
  4   -7.852646677089       -6.17       -3.34    3.0
  5   -7.852646685926       -8.05       -4.77    1.0
  6   -7.852646686727       -9.10       -5.26    3.8
  7   -7.852646686730      -11.55       -5.94    1.5
  8   -7.852646686730      -12.53       -7.36    2.2
  9   -7.852646686730      -15.05       -7.66    2.5
 10   -7.852646686730   +  -14.75       -8.40    2.5
 11   -7.852646686730      -14.45       -9.86    2.8
 12   -7.852646686730   +  -14.75      -10.42    2.5
 13   -7.852646686730      -14.75      -11.56    1.8
 14   -7.852646686730   +    -Inf      -11.76    2.0
 15   -7.852646686730   +  -14.75      -11.86    1.0
 16   -7.852646686730   +  -15.05      -11.57    1.0
 17   -7.852646686730      -15.05      -12.04 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846886050782                   -0.70           4.0
  2   -7.852527981574       -2.25       -1.62   0.80    5.0
  3   -7.852610176980       -4.09       -2.80   0.80    5.2
  4   -7.852646567048       -4.44       -3.55   0.80    7.2
  5   -7.852646683352       -6.93       -4.44   0.80    8.5
  6   -7.852646686647       -8.48       -5.06   0.80   10.2
  7   -7.852646686725      -10.11       -6.30   0.80   11.5
  8   -7.852646686730      -11.29       -7.26   0.80   13.5
  9   -7.852646686730      -13.77       -7.64   0.80   15.0
 10   -7.852646686730   +  -14.57       -9.15   0.80   15.5
 11   -7.852646686730   +    -Inf       -9.26   0.80   19.0
 12   -7.852646686730      -14.75      -10.48   0.80   18.8
 13   -7.852646686730   +    -Inf      -11.26   0.80   21.0
 14   -7.852646686730      -14.75      -11.90   0.80   22.2
 15   -7.852646686730   +  -14.57      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.361117e+01     3.437960e+00
 * time: 0.5520851612091064
     1     1.193163e+00     2.179569e+00
 * time: 0.8275530338287354
     2    -1.767455e+00     2.534029e+00
 * time: 0.8591110706329346
     3    -3.724458e+00     2.068348e+00
 * time: 0.9021730422973633
     4    -5.109318e+00     1.847404e+00
 * time: 0.9449090957641602
     5    -6.703593e+00     1.419408e+00
 * time: 0.993610143661499
     6    -7.450731e+00     8.112161e-01
 * time: 1.0427980422973633
     7    -7.691770e+00     6.416081e-01
 * time: 1.0730769634246826
     8    -7.798857e+00     9.025425e-02
 * time: 1.1036961078643799
     9    -7.825772e+00     1.980489e-01
 * time: 1.1347360610961914
    10    -7.840404e+00     9.200451e-02
 * time: 1.1651639938354492
    11    -7.847557e+00     4.469908e-02
 * time: 1.1954460144042969
    12    -7.851113e+00     2.301258e-02
 * time: 1.2258460521697998
    13    -7.851996e+00     2.163339e-02
 * time: 1.25631999969

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846826101947                   -0.70    4.0
  2   -7.852296168992       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686064                   -2.56
  2   -7.852646686730       -9.18       -5.74
  3   -7.852646686730   +    -Inf      -10.97
  4   -7.852646686730   +    -Inf      -15.98
|ρ_newton - ρ_scf|  = 1.2488565188344057e-13
|ρ_newton - ρ_scfv| = 9.296881831225504e-14
|ρ_newton - ρ_dm|   = 4.780651580598296e-10
